---
# Запуск модели для распознания именованных сущностей

> **🎯 Цель этого ноутбука:** провести распознавание именованных сущностей в собранном датасете при помощи кастомной NER модели.
---

## Импорт библиотек

In [ ]:
import pandas as pd
import torch
from transformers import pipeline
from tqdm.auto import tqdm

## Инференс

In [ ]:
SOURCE_DATA_PATH = "../data/got_q_db.parquet"
OUTPUT_DATA_PATH = "../data/chgk_database_with_entities.parquet"
MODEL_NAME = "borisMI/ChGK_NER"
BATCH_SIZE = 16

device = 0 if torch.cuda.is_available() else -1
if device == 0:
    print(f"Используется GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Используется CPU. Обработка может занять много времени.")

print(f"\nЗагрузка данных из {SOURCE_DATA_PATH}...")
df = pd.read_parquet(SOURCE_DATA_PATH)
print("Данные загружены.")

print("\nСоздание колонки 'full_text'...")
df['question'] = df['text'].fillna('')
df['answer'] = df['answer'].fillna('')
df['comments'] = df['comment'].fillna('')
df['full_text'] = df['question'] + ' ' + df['answer'] + ' ' + df['comments']
print("Колонка 'full_text' создана.")


print(f"\nЗагрузка NER-модели '{MODEL_NAME}'...")
ner_pipeline = pipeline(
    "ner",
    model=MODEL_NAME,
    tokenizer=MODEL_NAME,
    aggregation_strategy="simple",
    device=device
)
print("Модель успешно загружена.")
tokenizer = ner_pipeline.tokenizer
max_length = 512

texts_to_process = df['full_text'].tolist()

print(f"\nПодготовка и обрезка текстов до максимальной длины ({max_length} токенов)...")
truncated_texts = []
for text in tqdm(texts_to_process, desc="Обрезка текстов"):
    tokens = tokenizer.encode(text, truncation=True, max_length=max_length)
    truncated_text = tokenizer.decode(tokens, skip_special_tokens=True)
    truncated_texts.append(truncated_text)

print("Обрезка завершена.")

In [ ]:
print(f"\nНачинаем извлечение сущностей. Всего текстов: {len(truncated_texts)}.")

all_entities = []
for i in tqdm(range(0, len(truncated_texts), BATCH_SIZE), desc="Обработка батчей"):

    batch_texts = truncated_texts[i:i + BATCH_SIZE]
    results = ner_pipeline(batch_texts)
    all_entities.extend(results)

print("Извлечение сущностей завершено.")

# Добавляем результаты в DataFrame
df['entities'] = all_entities

In [ ]:
context_columns = ['id', 'author', 'date', 'entities']

existing_context_columns = [col for col in context_columns if col in df.columns]
print(f"\nВыбираем колонки для контекста: {existing_context_columns}")

df_subset = df[existing_context_columns]
df_exploded = df_subset.explode('entities').dropna(subset=['entities']).reset_index(drop=True)
entity_details = pd.json_normalize(df_exploded['entities'])
df_final = df_exploded.join(entity_details)
df_final = df_final.drop(columns=['entities'])

if 'playedAt' in df_final.columns:
    df_final['year'] = pd.to_datetime(df_final['playedAt'], errors='coerce').dt.year

print("\nИтоговый структурированный DataFrame (каждая сущность в своей строке):")
print(df_final.info())
print(df_final.head())

OUTPUT_FILE = "chgk_entities_structured.parquet"
df_final.to_parquet(OUTPUT_FILE, index=False)
print(f"\nЧистые структурированные данные сохранены в файл: {OUTPUT_FILE}")
